In [ ]:
import os
from matplotlib import pylab as plt
from os import path
from numpy import array, maximum, minimum, median
import numpy as np

### Set plotting themes

In [ ]:
%matplotlib inline
plt.style.use(['dark_background', 'bmh'])
plt.rc('axes', facecolor='k')
plt.rc('figure', facecolor='k')
plt.rc('figure', figsize=(20,5))

### Set plotting globals

In [ ]:
N = 24
seeds = [i for i in range(1, 4)]
steps = [(i + 1) * 5000 for i in range(N)]
tls = [1.0]
lls = [0.0, 0.1, 0.2]

### Save target lane results

In [ ]:
success_tl = list(list(list(list() for seed in seeds) for tl in tls) for ll in lls)
performance_path_tl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/models_learned_cost/planning_results'
name_tl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal={ll}' + \
       '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=False-learnedcost=True-lambdatl={tl}' + \
       '-seed={seed}-novaluestep{step}.model.log'

for i, ll in enumerate(lls):
    for j, tl in enumerate(tls):
        for seed in seeds:
            for step in steps:
                file_name = path.join(performance_path_tl, name_tl.format(ll=ll, tl=tl, seed=seed, step=step))
                if os.path.isfile(file_name):
                    with open(file_name) as f:
                        success_tl[i][j][seed - 1].append(float(f.readlines()[-1].split()[-1]))
                else:
                    success_tl[i][j][seed - 1].append(None)

### Plot target lane results

In [ ]:
# Plot target lane results
if success_tl[0][0][0][0] is None:
    for i, ll in enumerate(lls):
        for j, tl in enumerate(tls):
            for seed in seeds: 
                plt.plot(steps[1::2], success_tl[i][j][seed - 1][1::2], label=f'll:{ll}; tl:{tl}; seed: {seed}')
else:
    for i, ll in enumerate(lls):
        for j, tl in enumerate(tls):
            for seed in seeds:
                plt.plot(steps, success_tl[i][j][seed - 1], label=f'll:{ll}; tl:{tl}; seed: {seed}')
plt.grid(True)
plt.xlabel('steps')
plt.ylabel('success rate')
plt.title('Success rates for varying lane cost lambdas (ll) and target lane lambdas (tl)')
plt.legend()
plt.ylim([0.50, 0.85])
plt.xlim([5_000, 120_000]);

### Save baseline results

In [ ]:
# performance_path_bl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/' + \
#                       'models_baseline/planning_results'
performance_path_bl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/models_v13/planning_results'
name_bl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal=0.2' + \
          '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=0-learnedcost=False' + \
          '-seed={seed}-novaluestep{step}.model.log'
success_bl = list(list() for seed in seeds)
for seed in seeds:
    for step in steps:
        file_name = path.join(performance_path_bl, name_bl.format(seed=seed, step=step))
        if os.path.isfile(file_name):
            with open(file_name) as f:
                success_bl[seed - 1].append(float(f.readlines()[-1].split()[-1]))
        else:
            success_bl[seed - 1].append(None)

### Plot target lane vs. baseline results

In [ ]:
# Plot baseline
for seed in seeds:
    plt.plot(
        array(steps) / 1e3, success_bl[seed - 1],
        label=f'baseline; seed: {seed}',
        alpha=1,
        linewidth=4
    )
# Plot target lane
for i, ll in enumerate(lls):
    for j, tl in enumerate(tls):
        for seed in seeds:
            plt.plot(
                array(steps) / 1e3, success_tl[i][j][seed - 1],
                label=f'll: {ll}; tl: {tl}; seed: {seed}',
                alpha=0.4,
                linewidth=1
            )
plt.grid(True)
plt.xlabel('steps [k]')
plt.ylabel('success rate')
plt.legend(ncol=7)
plt.ylim([0.3, 0.85])
plt.xlim([5, 120])
plt.title('Baseline vs. Target Lane policy success rate')
plt.xticks(range(10, 100 + 20, 10));

# plt.savefig('Stc-vs-dtr-success_rate.png', bbox_inches = 'tight')

In [ ]:
# # Gather baseline plot elements
# names = ['bl']
# N_arr = {'bl': min([sum(x is not None for x in sbl) for sbl in success_bl])}
# steps_arr = {'bl': steps[:N_arr['bl']]}
# success_arr = {'bl': array([sbl[:N_arr['bl']] for sbl in success_bl])}

# # Gather target lane plot elements
# for i, ll in enumerate(lls):
#     for j, tl in enumerate(tls):
#         name = f'll-{ll}_tl-{tl}'
#         names.append(name)
#         N_arr[name] = min([sum(x is not None for x in stl) for stl in success_tl[i][j]])
#         steps_arr[name] = steps[:N_arr[name]]
#         success_arr[name] = array([stl[:N_arr[name]] for stl in success_tl[i][j]])
# # stats = ('min', 'max', 'median')
# for k in names:
#     plt.plot(
#         array(steps_arr[k]) / 1e3, np.median(success_arr[k], axis=0),
#         label=f'{k}',
#         linewidth=2,
#     )
# for k in names:
#     plt.fill_between(
#         array(steps_arr[k]) / 1e3, success_arr[k].min(0), success_arr[k].max(0),
#         alpha=.5,
#     )
# plt.grid(True)
# plt.xlabel('steps [k–]')
# plt.ylabel('success rate')
# plt.legend(ncol=7)
# plt.ylim([0.50, 0.85])
# plt.xlim([5, 120])
# plt.title('Baseline vs. Target Lane policy success rate min-max')
# plt.xticks(range(10, 100 + 20, 10));

# # plt.savefig('Rgr-vs-hrd-success_rate-min-max.png', bbox_inches = 'tight')

## Parse logs to find out how we did in instances where car switched lanes

### Baseline

In [ ]:
baseline_logs_dir = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/logs/' + \
                    'target_lane_learned_cost/baseline/planning_results'
log_bl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal=0.2' + \
         '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=0-learnedcost=False' + \
         '-seed={seed}-novaluestep{step}.model.out'
N_bl = list(list() for seed in seeds)
time_eps_bl = list(list(list() for step in range(len(steps))) for seed in seeds)
distance_eps_bl = list(list(list() for step in range(len(steps))) for seed in seeds)
success_eps_bl = list(list(list() for step in range(len(steps))) for seed in seeds)
lane_change_eps_bl = list(list(list() for step in range(len(steps))) for seed in seeds)

for seed in seeds:
    N = 0
    for s, step in enumerate(steps):
        file_name = path.join(baseline_logs_dir, log_bl.format(seed=seed, step=step))
        if os.path.isfile(file_name):
            N += 1
            with open(file_name) as f:
                lines = f.readlines()
                for i in range(len(lines)):
                    if 'new episode' in lines[i]:
                        while 'cntr' not in lines[i]:
                            i += 1
                        current_lane = float(lines[i].split()[3].strip())
                        target_lane = float(lines[i].split()[-1].strip())
                        lane_change_eps_bl[seed-1][s].append(abs(current_lane - target_lane) >= 24.0)
                        while 'mean success' not in lines[i]:
                            i += 1
                        time = float(lines[i].split('|')[1].split(':')[1].strip())
                        distance = float(lines[i].split('|')[2].split(':')[1].strip())
                        success = int(lines[i].split('|')[3].split(':')[1].strip())
                        time_eps_bl[seed-1][s].append(time)
                        distance_eps_bl[seed-1][s].append(distance)
                        success_eps_bl[seed-1][s].append(success)
    N_bl[seed-1].append(N)

In [ ]:
# Save Results
avg_time_bl = list(list() for seed in seeds)
avg_distance_bl = list(list() for seed in seeds)
avg_success_bl = list(list() for seed in seeds)
for seed in seeds:
    for s in range(N_bl[seed-1][0]):
        times = np.array(time_eps_bl[seed-1][s])[np.array(lane_change_eps_bl[seed-1][s])]
        distances = np.array(distance_eps_bl[seed-1][s])[np.array(lane_change_eps_bl[seed-1][s])]
        successes = np.array(success_eps_bl[seed-1][s])[np.array(lane_change_eps_bl[seed-1][s])]
        avg_time_bl[seed-1].append(np.mean(times))
        avg_distance_bl[seed-1].append(np.mean(distances))
        avg_success_bl[seed-1].append(np.mean(successes))

### With target lane

In [ ]:
tl_logs_dir = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/logs/' + \
              'target_lane_learned_cost/planning_results'
log_tl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal={ll}' + \
         '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=False-learnedcost=True-lambdatl={tl}' + \
         '-seed={seed}-novaluestep{step}.model.out'
N_tl = list(list(list(list() for seed in seeds) for tl in tls) for ll in lls)
time_eps_tl = list(list(list(list(list() for step in range(len(steps))) for seed in seeds) for tl in tls)
                   for ll in lls)
distance_eps_tl = list(list(list(list(list() for step in range(len(steps))) for seed in seeds) for tl in tls)
                       for ll in lls)
success_eps_tl = list(list(list(list(list() for step in range(len(steps))) for seed in seeds) for tl in tls)
                      for ll in lls)
lane_change_eps_tl = list(list(list(list(list() for step in range(len(steps))) for seed in seeds) for tl in tls)
                          for ll in lls)

for l, ll in enumerate(lls):
    for t, tl in enumerate(tls):
        for seed in seeds:
            N = 0
            for s, step in enumerate(steps):
                file_name = path.join(tl_logs_dir, log_tl.format(ll=ll, tl=tl, seed=seed, step=step))
                if os.path.isfile(file_name):
                    N += 1
                    with open(file_name) as f:
                        lines = f.readlines()
                        for i in range(len(lines)):
                            if 'new episode' in lines[i]:
                                while 'cntr' not in lines[i]:
                                    i += 1
                                current_lane = float(lines[i].split()[3].strip())
                                target_lane = float(lines[i].split()[-1].strip())
                                lane_change_eps_tl[l][t][seed-1][s].append(abs(current_lane - target_lane) >= 24.0)
                                while 'mean success' not in lines[i]:
                                    i += 1
                                time = float(lines[i].split('|')[1].split(':')[1].strip())
                                distance = float(lines[i].split('|')[2].split(':')[1].strip())
                                success = int(lines[i].split('|')[3].split(':')[1].strip())
                                time_eps_tl[l][t][seed-1][s].append(time)
                                distance_eps_tl[l][t][seed-1][s].append(distance)
                                success_eps_tl[l][t][seed-1][s].append(success)
            N_tl[l][t][seed-1].append(N)

In [ ]:
# Save Results
avg_time_tl = list(list(list(list() for seed in seeds) for tl in tls) for ll in lls)
avg_distance_tl = list(list(list(list() for seed in seeds) for tl in tls) for ll in lls)
avg_success_tl = list(list(list(list() for seed in seeds) for tl in tls) for ll in lls)
for l, ll in enumerate(lls):
    for t, tl in enumerate(tls):
        for seed in seeds:
            for s in range(N_tl[l][t][seed-1][0]):
                times = np.array(time_eps_tl[l][t][seed-1][s])[np.array(lane_change_eps_tl[l][t][seed-1][s])]
                distances = np.array(distance_eps_tl[l][t][seed-1][s])[np.array(lane_change_eps_tl[l][t][seed-1][s])]
                successes = np.array(success_eps_tl[l][t][seed-1][s])[np.array(lane_change_eps_tl[l][t][seed-1][s])]
                avg_time_tl[l][t][seed-1].append(np.mean(times))
                avg_distance_tl[l][t][seed-1].append(np.mean(distances))
                avg_success_tl[l][t][seed-1].append(np.mean(successes))

## Plot comparison

In [ ]:
# Plot results: Success
for seed in seeds:
    plt.plot(
        steps[:N_bl[seed-1][0]], avg_success_bl[seed - 1],
        label=f'baseline; seed: {seed}',
        alpha=1,
        linewidth=4
    )
for l, ll in enumerate(lls):
    for t, tl in enumerate(tls):
        for seed in seeds:
            plt.plot(steps[:N_tl[l][t][seed-1][0]], avg_success_tl[l][t][seed - 1],
                     label=f'll:{ll}; tl:{tl}; seed: {seed}', alpha=1, linewidth=2
                    )
plt.grid(True)
plt.xlabel('steps')
plt.ylabel('success rate (survival in correct lane)')
plt.title('Success rates for varying lane cost lambdas (ll) and target lane lambdas (tl) compared to baseline')
plt.legend()
plt.xlim([5_000, 120_000])
plt.show();

# Plot results: Time
for seed in seeds:
    plt.plot(
        steps[:N_bl[seed-1][0]], avg_time_bl[seed - 1],
        label=f'baseline; seed: {seed}',
        alpha=1,
        linewidth=4
    )
for l, ll in enumerate(lls):
    for t, tl in enumerate(tls):
        for seed in seeds:
            plt.plot(steps[:N_tl[l][t][seed-1][0]], avg_time_tl[l][t][seed - 1],
                     label=f'll:{ll}; tl:{tl}; seed: {seed}')
plt.grid(True)
plt.xlabel('steps')
plt.ylabel('Avg Drive Time')
plt.title('Avg Drive Times for varying lane cost lambdas (ll) and target lane lambdas (tl) compared to baseline')
plt.legend()
plt.xlim([5_000, 120_000])
plt.show();

# Plot results: Distance
for seed in seeds:
    plt.plot(
        steps[:N_bl[seed-1][0]], avg_distance_bl[seed - 1],
        label=f'baseline; seed: {seed}',
        alpha=1,
        linewidth=4
    )
for l, ll in enumerate(lls):
    for t, tl in enumerate(tls):
        for seed in seeds:
            plt.plot(steps[:N_tl[l][t][seed-1][0]], avg_distance_tl[l][t][seed - 1],
                     label=f'll:{ll}; tl:{tl}; seed: {seed}')
plt.grid(True)
plt.xlabel('steps')
plt.ylabel('Avg Drive Distance')
plt.title('Avg Drive Distances for varying lane cost lambdas (ll) and target lane lambdas (tl) compared to baseline')
plt.legend()
plt.xlim([5_000, 120_000])
plt.show();

In [ ]:
# Gather baseline plot elements
names = ['bl']
N_arr = {'bl': min([n[0] for n in N_bl])}
steps_arr = {'bl': steps[:N_arr['bl']]}
success_arr = {'bl': array([sbl[:N_arr['bl']] for sbl in avg_success_bl])}
time_arr = {'bl': array([tbl[:N_arr['bl']] for tbl in avg_time_bl])}
distance_arr = {'bl': array([dbl[:N_arr['bl']] for dbl in avg_distance_bl])}

# Gather target lane plot elements
for l, ll in enumerate(lls):
    for t, tl in enumerate(tls):
        name = f'll-{ll}_tl-{tl}'
        names.append(name)
        N_arr[name] = min([n[0] for n in N_tl[l][t]])
        steps_arr[name] = steps[:N_arr[name]]
        success_arr[name] = array([stl[:N_arr[name]] for stl in avg_success_tl[l][t]])
        time_arr[name] = array([ttl[:N_arr[name]] for ttl in avg_time_tl[l][t]])
        distance_arr[name] = array([dtl[:N_arr[name]] for dtl in avg_distance_tl[l][t]])

        
# Plot comparison: Survival        
for k in names:
    plt.plot(
        array(steps_arr[k]) / 1e3, np.median(success_arr[k], axis=0),
        label=f'{k}',
        linewidth=2,
    )
for k in names:
    plt.fill_between(
        array(steps_arr[k]) / 1e3, success_arr[k].min(0), success_arr[k].max(0),
        alpha=.5,
    )
plt.grid(True)
plt.xlabel('steps [k–]')
plt.ylabel('success rate (survival in correct lane)')
plt.legend(ncol=7)
plt.xlim([5, 120])
plt.title('Baseline vs. Target Lane policy success rate min-max')
plt.xticks(range(10, 100 + 20, 10))
plt.show();

# Plot comparison: Time        
for k in names:
    plt.plot(
        array(steps_arr[k]) / 1e3, np.median(time_arr[k], axis=0),
        label=f'{k}',
        linewidth=2,
    )
for k in names:
    plt.fill_between(
        array(steps_arr[k]) / 1e3, time_arr[k].min(0), time_arr[k].max(0),
        alpha=.5,
    )
plt.grid(True)
plt.xlabel('steps [k–]')
plt.ylabel('Avg Drive Time')
plt.legend(ncol=7)
plt.xlim([5, 120])
plt.ylim(100, 350)
plt.title('Baseline vs. Target Lane policy Avg Drive Time min-max')
plt.xticks(range(10, 100 + 20, 10))
plt.show();

# Plot comparison: Survival        
for k in names:
    plt.plot(
        array(steps_arr[k]) / 1e3, np.median(distance_arr[k], axis=0),
        label=f'{k}',
        linewidth=2,
    )
for k in names:
    plt.fill_between(
        array(steps_arr[k]) / 1e3, distance_arr[k].min(0), distance_arr[k].max(0),
        alpha=.5,
    )
plt.grid(True)
plt.xlabel('steps [k–]')
plt.ylabel('Avg Drive Distance')
plt.legend(ncol=7)
plt.xlim([5, 120])
plt.ylim(500, 1100)
plt.title('Baseline vs. Target Lane policy Avg Drive Distance min-max')
plt.xticks(range(10, 100 + 20, 10))
plt.show();
